In [28]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from urllib.parse import quote
import time

def get_coupang_search_results(search_keyword: str, advanced_keywords: list[str]):
    encoded_keyword = quote(search_keyword)
    url = f"https://www.tw.coupang.com/search?q={encoded_keyword}&channel=user"

    options = Options()
    # 若需隱藏瀏覽器可啟用
    options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--lang=zh-TW")

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    driver.get(url)
    time.sleep(5)

    html = driver.page_source
    # 若需重複操作瀏覽器可先保留不關閉
    driver.quit()

    soup = BeautifulSoup(html, "html.parser")
    product_cards = soup.select("div.SearchResult_searchResultProduct___h6E9")

    results = []
    matched_results = []

    for card in product_cards:
        try:
            # 完整內容文字
            full_text = card.get_text(separator=" ", strip=True)

            # 精簡標題（僅取 title 區塊）
            title_tag = card.select_one("div.Product_title__8K0xk")
            title = title_tag.get_text(strip=True) if title_tag else "N/A"

            # 價格
            price_tag = card.select_one("span.Product_salePricePrice__2FbsL span")
            price = price_tag.get_text(strip=True) if price_tag else "N/A"

            # 每單位價格
            unit_price_tag = card.select_one("div.Product_unitPrice__QQPdR")
            unit_price = unit_price_tag.get_text(strip=True) if unit_price_tag else "N/A"

            product = {
                "title": title,
                "full_text": full_text,
                "price": price,
                "unit_price": unit_price
            }

            results.append(product)

            # 進階條件：從完整描述中過濾
            if all(kw in full_text for kw in advanced_keywords):
                matched_results.append(product)

        except Exception as e:
            print("解析錯誤：", e)
            continue

    return results, matched_results

# 主程式區塊
if __name__ == "__main__":
    # 搜尋主關鍵字
    search_keyword = "DOG CAT STAR 汪喵星球"

    # 進階篩選條件（需同時包含全部關鍵詞）
    advanced_keywords = ["米粒型"]

    # 執行爬取
    all_products, filtered_products = get_coupang_search_results(search_keyword, advanced_keywords)

    print(f"搜尋關鍵字：{search_keyword}")
    print("所有搜尋結果：")
    for idx, product in enumerate(all_products, 1):
        print(f"{idx}. 標題: {product['title']}")
        print(f"   價格: {product['price']}")
        print(f"   每單位: {product['unit_price']}")
        print(f"   完整內容: {product['full_text']}")
        print("-" * 60)

    print(f"\n進階條件符合項目（包含：{'、'.join(advanced_keywords)}）：")
    for idx, product in enumerate(filtered_products, 1):
        print(f"{idx}. 標題: {product['title']}")
        print(f"   價格: {product['price']}")
        print(f"   每單位: {product['unit_price']}")
        print(f"   完整內容: {product['full_text']}")
        print("-" * 60)


搜尋關鍵字：DOG CAT STAR 汪喵星球
所有搜尋結果：
1. 標題: DOG CAT STAR 汪喵星球 益生菌消臭條型豆腐砂, 2.7kg, 6袋
   價格: $857
   每單位: ($5.29/100g)
   完整內容: DOG CAT STAR 汪喵星球 益生菌消臭條型豆腐砂, 2.7kg, 6袋 特價 41折 $2,100 $857 ($5.29/100g) 7折 優惠券 明天 5/12 (一) 預計送達 免運 ( 2,633 )
------------------------------------------------------------
2. 標題: DOG CAT STAR 汪喵星球 益生菌消臭豆腐砂 米粒型, 2.7kg, 6袋
   價格: $935
   每單位: ($5.77/100g)
   完整內容: DOG CAT STAR 汪喵星球 益生菌消臭豆腐砂 米粒型, 2.7kg, 6袋 特價 91折 $1,035 $935 ($5.77/100g) 7折 優惠券 明天 5/12 (一) 預計送達 免運 ( 11 )
------------------------------------------------------------
3. 標題: DOG CAT STAR 汪喵星球 冷凍乾燥小零嘴 犬貓適用, 雞胸肉, 50g, 1包
   價格: $116
   每單位: ($23.20/10g)
   完整內容: DOG CAT STAR 汪喵星球 冷凍乾燥小零嘴 犬貓適用, 雞胸肉, 50g, 1包 特價 53折 $220 $116 ($23.20/10g) 7折 優惠券 明天 5/12 (一) 預計送達 免運 滿 $490 ( 2,964 )
------------------------------------------------------------
4. 標題: MAOUP 毛起來 毛孩每日健康賞 呼吸道養氣賞 草本複方, 30包, 氣管健康保養, 1袋
   價格: $375
   每單位: N/A
   完整內容: MAOUP 毛起來 毛孩每日健康賞 呼吸道養氣賞 草本複方, 30包, 氣管健康保養, 1袋 特價 73折 $519 $375 7折 優惠券 明天 5/12 (一) 預計送達 